![Banner API](<../../5. Sources/Images/banner_api.gif>)

## Extracción API HotelBeds

In [6]:
import requests
import json
import time, hashlib
import keys as k
import pandas as pd

In [7]:
# Tu API Key y Secret proveniente de keys el cual está en .gitignore
apiKey = k.apiKey_t
Secret = k.Secret_t

# Signature es generado por SHA256 (Api-Key + Secret + Timestamp (en segundos))
sigStr = "%s%s%d" % (apiKey, Secret, int(time.time()))
signature = hashlib.sha256(sigStr.encode()).hexdigest()

### Test de conexión de datos tipo POST para hotelbeds

### Consulta de disponibilidad del Hotel en tiempo real

Esta conexión nos extrae datos de los hoteles según la geolocalización y parametros de distancia en forma radial o en forma de rectangulo pero no es beneficioso, por lo cual es necesario implementar la extracción por id de hotel y configurar un bucle el cual haga el proceso por lotes de 2000 registros, entonces se necesita la data extraida por medio de GET.

In [6]:
# URL de conexión modo test para hoteles y disponibilidad

url = "https://api.test.hotelbeds.com/hotel-api/1.0/hotels"

payload = json.dumps({
  "stay": {
    "checkIn": "2023-09-30",
    "checkOut": "2023-10-01",
    "shiftDays": 1
  },
  "geolocation": {
    "longitude": "-116.3524076",
    "latitude": "38.8314452",
    "radius": "200",
    "unit": "mi"
  },
  "occupancies": [
    {
      "rooms": 1,
      "adults": 2,
      "children": 0
    },
    {
      "rooms": 1,
      "adults": 1,
      "children": 0
    },
    {
      "rooms": 1,
      "adults": 1,
      "children": 1,
      "paxes": [
        {
          "type": "CH",
          "age": 2
        }
      ]
    }
  ]
})
headers = {
  'Api-key': apiKey,
  'X-Signature': signature,
  'Accept': 'application/json',
  'Accept-Encoding': 'gzip',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

# Verifica que la solicitud fue exitosa (código de respuesta 200)
if response.status_code == 200:
    # Convierte la respuesta JSON en un diccionario de Python
    data = json.loads(response.text)

    # Especifica el nombre del archivo en el que deseas guardar los datos JSON
    archivo_json = "rt_hotel_occupancy_api.json"

    # Especifica la ruta donde se encuentra el archivo
    path_json = "../../2. Datasets/original/hotelbeds/"

    # Abre el archivo en modo escritura y guarda el diccionario en formato JSON
    with open(path_json + archivo_json, "w") as archivo:
        json.dump(data, archivo, indent=4)

    print(f"Los datos se han guardado en {archivo_json}")
else:
    print(f"Error al hacer la solicitud a la API. Código de respuesta: {response.status_code}")


Los datos se han guardado en rt_hotel_occupancy_api.json


### Test de conexión de datos tipo GET para hotelbeds

Primer código de implementación, pruebas de Conexión para 1000 registros

In [10]:
url = "https://api.test.hotelbeds.com/hotel-content-api/1.0/hotels"

query_params = {
  "countryCode": "US",
  "from": 1,
  "to": 1000
}

headers = {
  'Api-key': apiKey,
  'X-Signature': signature,
  'Accept': 'application/json',
  'Accept-Encoding': 'gzip',
  'Content-Type': 'application/json'
}

# Realiza la solicitud GET
response = requests.get(url, params=query_params, headers=headers)

# Verifica que la solicitud fue exitosa (código de respuesta 200)
if response.status_code == 200:
    # Convierte la respuesta JSON en un diccionario de Python
    data = json.loads(response.text)

    # Especifica el nombre del archivo en el que deseas guardar los datos JSON
    archivo_json = "test_hotel_api.json"

    # Especifica la ruta donde se encuentra el archivo
    path_json = "../../2. Datasets/original/hotelbeds/"

    # Abre el archivo en modo escritura y guarda el diccionario en formato JSON
    with open(path_json + archivo_json, "w") as archivo:
        json.dump(data, archivo, indent=4)

    print(f"Los datos se han guardado en {archivo_json}")
else:
    print(f"Error al hacer la solicitud a la API. Código de respuesta: {response.status_code}")

Los datos se han guardado en test_hotel_api.json


Se realiza extracción por medio de bucle con lotes de 1000 registros lo cual es lo máximo permitido por la API, se filtran los datos por país y se guarda por lotes ya que maneja bastante volumen de datos y evitar problemas de memoria.

In [4]:
# URL base de la API ti
url_base = "https://api.test.hotelbeds.com/hotel-content-api/1.0/hotels"

# Parámetros iniciales filtro por "US" inicial del país
query_params = {
  "countryCode": "US",
}

headers = {
  'Api-key': apiKey,
  'X-Signature': signature,
  'Accept': 'application/json',
  'Accept-Encoding': 'gzip',
  'Content-Type': 'application/json'
}

# Valor inicial y tamaño del lote
from_value = 1
batch_size = 1000
to_value = from_value + batch_size - 1

# Contador de archivos
file_count = 1

# Realiza solicitudes en lotes hasta llegar al valor "to"
while from_value <= 39963:
    # Actualiza los parámetros de consulta con los valores actuales de "from" y "to"
    query_params["from"] = from_value
    query_params["to"] = to_value

    # Realiza la solicitud GET
    response = requests.get(url_base, params=query_params, headers=headers)

    if response.status_code == 200:
        # Convierte la respuesta JSON en un diccionario de Python
        data = json.loads(response.text)

        # Especifica el nombre del archivo en el que deseas guardar los datos JSON totales
        archivo_json = f"hotels_us_total_api_{file_count}.json"

        # Especifica la ruta donde se encuentra el archivo
        path_json = "../../2. Datasets/original/hotelbeds/"

        # Abre el archivo en modo escritura y guarda el diccionario en formato JSON
        with open(path_json + archivo_json, "w") as archivo:
            json.dump(data["hotels"], archivo, indent=4)

        # Incrementa los valores de "from" y "to" para el próximo lote
        from_value += batch_size
        to_value += batch_size

        # Incrementa el contador
        file_count += 1

        
    else:
        print(f"Error al hacer la solicitud a la API. Código de respuesta: {response.status_code}")
        break
# disminuye el contador
file_count -= 1
print(f"Los datos se han guardado en {file_count} archivos .json")

Los datos se han guardado en 41 archivos .json


## Extracción del dataset de los detalles de los hoteles

se procede a extraer los detalles de los hoteles, es necesario obtener primero el dato del codigo del hotel a continuación el procedimiento:

In [9]:
# URL base de la API ti
url_base = "https://api.test.hotelbeds.com/hotel-content-api/1.0/hotels"

headers = {
  'Api-key': apiKey,
  'X-Signature': signature,
  'Accept': 'application/json',
  'Accept-Encoding': 'gzip',
  'Content-Type': 'application/json'
}


# Valor inicial y tamaño del lote
from_value = 1
batch_size = 200
to_value = from_value + batch_size - 1

# Contador de archivos
file_count = 1

# Inicializa el índice inicial
start = 0

# Inicializa una cadena vacía para almacenar los códigos del lote
codes_batch = ""

# Read the CSV file into a DataFrame
df_hotel_code = pd.read_csv('../../2. Datasets/original/hotelbeds/hotels_code.csv')

# Realiza solicitudes en lotes hasta llegar al valor "to"
while from_value <= 39963 - 1:
    
    # Calcula el índice final para el lote actual
    finish = start + batch_size
    
    # Extrae el lote actual de la columna 'code'
    batch = df_hotel_code['code'].iloc[start:finish]

    # Concatena los códigos del lote con comas y agrega al resultado anterior
    codes_batch += "," + ",".join(batch.astype(str))
    
    # Elimina la coma inicial si existe
    if codes_batch.startswith(","):
        codes_batch = codes_batch[1:]

    # Realiza la solicitud GET
    response = requests.get(url_base + f"/{codes_batch}/details", headers=headers)

    if response.status_code == 200:
        # Convierte la respuesta JSON en un diccionario de Python
        data = json.loads(response.text)

        # Especifica el nombre del archivo en el que deseas guardar los datos JSON totales
        archivo_json = f"hotels_us_details_api_{file_count}.json"

        # Especifica la ruta donde se encuentra el archivo
        path_json = "../../2. Datasets/original/hotelbeds/"

        # Abre el archivo en modo escritura y guarda el diccionario en formato JSON
        with open(path_json + archivo_json, "w") as archivo:
            json.dump(data, archivo, indent=4)

        # Incrementa los valores de "from" y "to" para el próximo lote
        from_value += batch_size
        to_value += batch_size

        # Incrementa el contador
        file_count += 1

        # Actualiza el índice inicial para el próximo lote
        start = finish
        
        # Se reestablece la variable que contiene la lista para crear una nueva
        codes_batch = ""

        
    else:
        print(f"Error al hacer la solicitud a la API. Código de respuesta: {response.status_code}")
        break
# disminuye el contador
file_count -= 1
print(f"Los datos se han guardado en {file_count} archivos .json")

Los datos se han guardado en 200 archivos .json
